## Deliverable 2. Create a Customer Travel Destinations Map.

In [39]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Tiksi,RU,71.6872,128.8694,-10.17,100,100,7.40,overcast clouds
1,1,Beringovskiy,RU,63.0500,179.3167,0.86,78,99,19.55,overcast clouds
2,2,Fuengirola,ES,36.5400,-4.6247,66.20,40,0,10.36,clear sky
3,3,Lompoc,US,34.6391,-120.4579,67.78,48,75,4.00,broken clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,-20.13,100,100,10.02,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
5,5,Port Alfred,ZA,-33.5906,26.8910,70.32,79,83,18.16,broken clouds
15,15,Rikitea,PF,-23.1203,-134.9692,77.72,75,100,16.71,light rain
18,18,Cockburn Town,TC,21.4612,-71.1419,75.38,74,62,23.76,broken clouds
21,21,San Ramon De La Nueva Oran,AR,-23.1322,-64.3260,75.54,85,92,4.29,overcast clouds
23,23,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,94,75,4.61,broken clouds
...,...,...,...,...,...,...,...,...,...,...
634,634,Ariquemes,BR,-9.9133,-63.0408,74.32,92,96,1.10,overcast clouds
637,637,Aloleng,PH,16.1307,119.7824,78.10,81,19,10.18,few clouds
640,640,Leon,MX,21.1167,-101.6667,77.83,22,14,16.53,few clouds
645,645,Gamba,GA,-2.6500,10.0000,74.82,88,100,0.58,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        169
City           169
Country        168
Lat            169
Lng            169
Max Temp       169
Humidity       169
Cloudiness     169
Wind Speed     169
Description    169
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
clean_df.count()

City_ID        168
City           168
Country        168
Lat            168
Lng            168
Max Temp       168
Humidity       168
Cloudiness     168
Wind Speed     168
Description    168
dtype: int64

In [40]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
5,Port Alfred,ZA,70.32,broken clouds,-33.5906,26.8910,NaN
15,Rikitea,PF,77.72,light rain,-23.1203,-134.9692,NaN
18,Cockburn Town,TC,75.38,broken clouds,21.4612,-71.1419,NaN
21,San Ramon De La Nueva Oran,AR,75.54,overcast clouds,-23.1322,-64.3260,NaN
23,Arraial Do Cabo,BR,75.16,broken clouds,-22.9661,-42.0278,NaN
31,Venice,US,79.11,haze,27.0998,-82.4543,NaN
32,Sibolga,ID,73.83,broken clouds,1.7427,98.7792,NaN
45,Sao Filipe,CV,74.21,clear sky,14.8961,-24.4956,NaN
46,Kedougou,SN,78.60,clear sky,12.5579,-12.1743,NaN
53,Kaeo,NZ,75.33,broken clouds,-35.1000,173.7833,NaN


In [41]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, city in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = city["Lat"]
    lng = city["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found...skipping.")           

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [42]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
5,Port Alfred,ZA,70.32,broken clouds,-33.5906,26.8910,The Halyards Hotel
15,Rikitea,PF,77.72,light rain,-23.1203,-134.9692,People ThankYou
18,Cockburn Town,TC,75.38,broken clouds,21.4612,-71.1419,Osprey Beach Hotel
21,San Ramon De La Nueva Oran,AR,75.54,overcast clouds,-23.1322,-64.3260,Hotel Residencial Centro
23,Arraial Do Cabo,BR,75.16,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
...,...,...,...,...,...,...,...
634,Ariquemes,BR,74.32,overcast clouds,-9.9133,-63.0408,Hotel Sol Nascente - Ariquemes - RO
637,Aloleng,PH,78.10,few clouds,16.1307,119.7824,Steve Walden and Emma Sison-Walden Residence
640,Leon,MX,77.83,few clouds,21.1167,-101.6667,ENTERPRISE INN POLIFORUM
645,Gamba,GA,74.82,overcast clouds,-2.6500,10.0000,NaN


In [43]:
len(hotel_df.iloc[166:167]["Hotel Name"])

1

In [44]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [45]:
clean_hotel_df.count()

City           158
Country        158
Max Temp       158
Description    158
Lat            158
Lng            158
Hotel Name     158
dtype: int64

In [46]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(20.0, 31.0), zoom_level =1.8)
marker_layer=gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))